In [2]:
import pygsheets
import pandas as pd

api_limit_size = 25
apikey_pairs = [
    { "a@gmail.com": "apikey" },
    { "b@gmail.com": "apikey" },
    { "c@gmail.com": "apikey" }
]

apikeys = [ list(key.values())[0] for key in apikey_pairs ]

gc = pygsheets.authorize(service_file='./news-project.json')
sht = gc.open_by_key('google-sheet_key')

df = sht.worksheet_by_title('Transactions').get_as_df(start="A2")
tickers_list = df[df["Asset Class"] == "Stock"]["Asset Ticker"].unique().tolist()

splits_list = [ tickers_list[i : i+api_limit_size] for i in range(0, len(tickers_list), api_limit_size) ]

print("apikeys: ", apikeys)
print("splits_list: ", splits_list)


# --------------------
# Widget Dropdown Tool

import ipywidgets as widgets
from IPython.display import display, clear_output

ticker_group_list = splits_list[0]
group_index = 0
def on_dropdown_change(change):
    global ticker_group_list
    global group_index
    clear_output()
    display(hbox)
    selected_index = change.new
    try:
        ticker_group_list = splits_list[selected_index]
    except IndexError as e:
        print(f"IndexError: splits_list does not contain index {selected_index}")
    group_index = selected_index
    print(f"ticker_group_list: {ticker_group_list}")

dropdown = widgets.Dropdown(
    options=[('0', 0), ('1', 1), ('2', 2)],
    value=0,
    # description='splits_list[selected_index]',
)
dropdown.observe(on_dropdown_change, names='value')

dropdown.layout.height = '30px'
dropdown.layout.width = '50px'

label = widgets.Label('splits_list[selected_index]')
label.layout.width = '200px'
label.layout.font_size = '24px'
label.layout.align_items = 'center'
label.layout.justify_content = 'center'

hbox = widgets.HBox([label, dropdown])

display(hbox)


In [100]:
ticker_group_list
group_index

1

In [102]:
import requests
import datetime

yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
yesterday_str = yesterday.strftime('%Y%m%d')

formatted_news = ''

apikey = apikeys[0]
for ticker in ticker_group_list:
    
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={apikey}&time_from={yesterday_str}T0600'
    try:
        response = requests.get(url).json()

        try:
            news_list = response["feed"]
        except KeyError as e:
            print(f"No feed found for {ticker}. Response: {response}\n")
            continue
        
        number = 1
        formatted_news += f"\n****************************************\nStocker Ticker {ticker} 總共有{len(news_list)}則新聞\n"
        for news in news_list:
            title = news["title"]
            url = news["url"]
            time_published = news["time_published"]
            summary = news["summary"]
            ticker_formatted_news = f"""
--------------------------------------------------
NO. {number}
Stock Ticker: {ticker}
URL: {url}
Published Time: {time_published}\n
Title: {title}\n
Summary: {summary} \n\n
"""
            formatted_news += ticker_formatted_news
            number += 1
    except requests.RequestException as e:
        print(f"Request fail for ticker {ticker}. Error is: {e}")
    except ValueError as e:
        print(f"Error decoding JSON response for ticker {ticker}. Error: {e}")
    except Exception as e:
        print(f"Error occurred: {e}")
        
print("Formatted_news: \n", formatted_news)

No feed found for IWM. Response: {'Information': 'Invalid inputs. Please refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}

Formatted_news: 
 
****************************************
Stocker Ticker SNAP 總共有0則新聞

****************************************
Stocker Ticker SNOW 總共有1則新聞

--------------------------------------------------
NO. 1
Stock Ticker: SNOW
URL: https://www.benzinga.com/insights/options/24/07/40048837/what-the-options-market-tells-us-about-snowflake
Published Time: 20240730T191612

Title: What the Options Market Tells Us About Snowflake - Snowflake  ( NYSE:SNOW ) 

Summary: Whales with a lot of money to spend have taken a noticeably bearish stance on Snowflake. Looking at options history for Snowflake SNOW we detected 40 trades. If we consider the specifics of each trade, it is accurate to state that 30% of the investors opened trades with bullish expectations and ... 



****************************************
Stocker Ti

In [103]:
# !mkdir stock_news
import os

today = datetime.datetime.now()
today_str = today.strftime("%Y-%m-%d")

dir_path = './stock_news'
file_path = f"{dir_path}/stockNews_{today_str}_{group_index}.txt"
file_name = os.path.basename(file_path)

with open(file_path, 'w') as file:
    file.write(formatted_news)
print(f"News has been saved to {dir_path}, filename: {file_name}")

News has been saved to ./stock_news, filename: stockNews_2024-07-31_1.txt


In [3]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# google api certification
SERVICE_ACCOUNT_FILE = "./news-project.json"
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/documents"]

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# 创建 Google Drive 和 Google Docs 服务
drive_service = build('drive', 'v3', credentials=credentials)
docs_service = build('docs', 'v1', credentials=credentials)

# 指定资料夹 ID 和文件名称
folder_id = "googleDrive_folder_id"
file_name = f"stockNews_{today_str}_{group_index}"


def create_google_docs_file(name, folder_id):
    file_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.document',
        'parents': [folder_id]
    }
    try:
        file = drive_service.files().create(body=file_metadata, fields="id").execute()
        print(f'Created file with ID: {file.get("id")}')
        return file.get('id')
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

def write_to_google_docs(doc_id, content):
    doc_metadata = [
        {
            'insertText': {
                'location': {
                    'index': 1
                },
                'text': content
            }
        }
    ]
    try:
        result = docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': doc_metadata}).execute()
        print(f'Updated document with ID: {doc_id}')
    except HttpError as error:
        print(f'An error occurred: {error}')

doc_id = create_google_docs_file(file_name, folder_id)
if doc_id:
    write_to_google_docs(doc_id, formatted_news)


In [106]:
list_formatted_news = formatted_news.split("****************************************")[1:]
len(list_formatted_news)

7

In [108]:
import ollama
import os
from tqdm import tqdm

ai_report_path = f"./ai_report/{today_str}_{group_index}.txt"

ai_num = 0

for oneTicker_content in tqdm(list_formatted_news, desc="Analyzing ticker news...", unit='B', unit_scale=True, unit_divisor=1024):
    
    ticker_prompt = f"""
你作為一個金融分析師和金融媒體專家，
請針對下面Stock Ticker所列新聞內容分別產出重點英文及繁體中文新聞摘要內容，
並在最下面列出所有新聞編號和URL，
如果Stock Ticker今天總共有0則新聞，沒有實際新聞內容，請直接回覆**今日沒有更新{{Stock Ticker}}新聞**，
請務必一開始就列出這篇摘要{{Stock Ticker}}為何，以及在最後列出我給你的原始新聞連結，

**產出格式如下**
[Stock Ticker] 新聞重點摘要如下:
英文摘要
1.第1項重點內容
2.第2項重點內容
......依序列出
參考新聞URL

中文摘要
1.第1項重點內容
2.第2項重點內容
......依序列出
參考新聞URL

所有原始新聞連結
1.URL1
2.URL2
......依序列出

**相關新聞內容如下**
{oneTicker_content}
"""
    
    response = ollama.chat(model='llama3.1:latest', messages=[
      {
        'role': 'user',
        'content': ticker_prompt,
      }], options={'temperature': 0.2, 'max_tokens': 500})
     
#     translate_prompt = f"""
# 請將下面英文摘要內容翻譯成繁體中文，
# 並只要提供給我繁體中文翻譯內容就好，
# 不要有其他內容

# **格式如下**
# [Stock Ticker] English Summary:
# translation content here
    
# 需要翻譯內容如下:
# {response['message']['content']}
# """
  
#     translate = ollama.chat(model='llama3.1:latest', messages=[
#       {
#         'role': 'user',
#         'content': translate_prompt,
#       },
#     ])


    result = f"""
\n
{response['message']['content']}\n
--------------------------------------------------
"""

#     result = f"""
# --------------------------------------------------
# 英文摘要\n
# {response['message']['content']}

# 中文摘要\n
# {translate['message']['content']}
# """
    print(str(ai_num) + "\n" + result)
    
    if os.path.exists(ai_report_path):
        with open(ai_report_path, "a") as file:
            file.write(result)
    else:
        with open(ai_report_path, "w") as file:
            file.write(result)


    ai_num += 1


Analyzing ticker news...:  14%|█████▌                                 | 1.00/7.00 [00:05<00:30, 5.13s/B]

0



**SNAP 新聞重點摘要如下:**
英文摘要
**今日沒有更新{SNAP}新聞**

中文摘要
**今日沒有更新{SNAP}新聞**

所有原始新聞連結
1. 
2. 
......依序列出

--------------------------------------------------



Analyzing ticker news...:  29%|███████████▏                           | 2.00/7.00 [00:11<00:28, 5.73s/B]

1



**Stock Ticker: SNOW**
新聞重點摘要如下:
英文摘要
1. Whales with a lot of money to spend have taken a noticeably bearish stance on Snowflake.
2. Looking at options history for Snowflake SNOW we detected 40 trades.

中文摘要
1. 大量資金的鯊魚已經對Snowflake表現出明顯的悲觀態度。
2. 對Snowflake選項歷史進行了檢視，我們發現了40個交易。

所有原始新聞連結:
1. https://www.benzinga.com/insights/options/24/07/40048837/what-the-options-market-tells-us-about-snowflake

--------------------------------------------------



Analyzing ticker news...:  43%|████████████████▋                      | 3.00/7.00 [00:34<00:54, 13.5s/B]

...

4



WOLF 新聞重點摘要如下:
英文摘要
**今日沒有更新WOLF新聞**

中文摘要
**今日沒有更新WOLF新聞**

所有原始新聞連結
1. 
2. 
......依序列出

--------------------------------------------------



Analyzing ticker news...:  86%|█████████████████████████████████▍     | 6.00/7.00 [00:48<00:07, 7.03s/B]

5



**Stock Ticker ACN新聞重點摘要如下：**
英文摘要
1. Microsoft's Q4 2024 earnings report shows a revenue increase of 12% year-over-year.
2. The company's operating income grew by 15% compared to the same period last year.

中文摘要
1.微軟Q4 2024財報顯示收入同比增長12%。
2.公司的營業收益較去年同期增長15%。

所有原始新聞連結：
1. https://www.fool.com/earnings/call-transcripts/2024/07/30/microsoft-msft-q4-2024-earnings-call-transcript/

--------------------------------------------------



Analyzing ticker news...: 100%|███████████████████████████████████████| 7.00/7.00 [01:06<00:00, 9.54s/B]

6



The articles provided are a list of news headlines from Business Standard on July 31, 2024. Here's a summary of the key points:

1. **Landslide in Wayanad**: The death toll has risen to 156 with many still trapped as rescue operations continue.
2. **Gold Trading**: Ahead of the US Fed outcome, check key levels for trading gold.
3. **Paris Olympics 2024**: India's schedule on July 31 will be live-streamed (IST).
4. **Google Autocomplete**: Results of Trump assassination bid raise questions.
5. **China's Factory Activity**: Shrinks in July with service sector growing more slowly.
6. **Silver Trading**: May consolidate gains before the Fed outcome.
7. **US Election**: Harris to hold a rally in Philadelphia on Tuesday with VP pick.
8. **Indices Opening**: Likely to open higher on Asian gains.
9. **Tata Motors Q1 Preview**: JLR and Indian CV sales may fuel revenue growth.
10. **Asian Stocks**: Steady, yen firm ahead of Bank of Japan's policy decision.
11. **Samsung Q2 Profit**: Soars o

In [109]:
import telebot

bot_token = 'telebot_token'
chat_id = 'telegram_chat_id'

bot = telebot.TeleBot(bot_token)

with open(f"./ai_report/{today_str}_{group_index}.txt", "r") as file:
    news_message = file.read()

list_news_message = news_message.split("--------------------------------------------------")[:-1]

bot_num = 0
for one_message in list_news_message:
    one_message = f"||{today_str} 新聞摘要||" + one_message
    if (len(one_message)<=4096):
        bot.send_message(chat_id, one_message)
    bot_num += 1

with open(f"./stock_news/stockNews_{today_str}_{group_index}.txt", 'rb') as file:
    bot.send_document(chat_id, file)